In [3]:
import os
from tqdm import tqdm

os.chdir('G:/내 드라이브/projects/NLP-StockMarket/model_fin/')

## 필요모듈 import

In [6]:
import scipy as sp
import pandas as pd
import numpy as np

from konlpy.tag import Kkma    
from konlpy.tag import Hannanum    
from konlpy.tag import Okt        
from konlpy.tag import *
import nltk
import pickle

# from wordcloud import WordCloud, STOPWORDS
from PIL import Image

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline
import os
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from sklearn.tree import export_graphviz
from datetime import datetime, timedelta

import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

In [3]:
kkma = Kkma()
hannanum = Hannanum()

## 자연어 데이터 import(날짜 가져오기 위해 먼저 가져옴)

In [15]:
# LG화학, 삼성SDI, SK이노베이션, 고려아연, 포스코케미칼
stock_name = '삼성SDI'

In [16]:
naver_news = pd.read_csv('./data/refined_naver_news.csv', index_col=0)
daum_news = pd.read_csv('./data/refined_daum_news.csv', index_col=0)
naver_talks = pd.read_csv(f'./data/refined_naver_talks_{stock_name}.csv', index_col=0)
daum_talks = pd.read_csv(f'./data/refined_daum_talks_{stock_name}.csv', index_col=0)
# youtube = pd.read_csv(f'./data/refined_youtube_{stock_name}.csv', index_col=0)

In [ ]:
talks = pd.concat([naver_talks, daum_talks], ignore_index=True)
for i in tqdm(range(len(talks))):
    talks['Date'][i] = str(talks['Date'][i]).replace('-', '').strip()

In [22]:
news_df = pd.concat([naver_news, daum_news, talks
#                     ,youtube
                    ], ignore_index=True)
news_df.head()

,Date,Title
0,20210101,서학개미 사로잡은 美 ESG ETF 올해도 高高할까
1,20210101,목표가 속속 100만원대 흥 오른 삼성바이오로직스
2,20210101,신생 PEF운용사 안타 떡볶이업체 SJ코레 사들여 1년만에 매각 80억 차익
3,20210101,삼성전자 신고가 랠리 9만전자 가능할까
4,20210101,작년 매출 신기록 쓴 카카오 엔씨 올해 더 좋다


## 주가 데이터

In [10]:
start = str(news_df.iloc[0, 0])
end = str(news_df.iloc[-1, 0])
print(start)
print(end)

20210101
20220630


### KRX에서 받은 csv로 가져오기

In [11]:
stock_df = pd.read_csv(f'./data/{stock_name}_주가_데이터.csv', index_col=0)
stock_df.head()

,일자,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
0,2022/06/30,532000,-38000,-6.67,564000,569000,532000,460818,248496695000,36582729960000,68764530
1,2022/06/29,570000,-11000,-1.89,571000,579000,569000,172702,98937812000,39195782100000,68764530
2,2022/06/28,581000,12000,2.11,568000,581000,565000,235259,135033955000,39952191930000,68764530
3,2022/06/27,569000,19000,3.45,563000,577000,561000,257213,146301753000,39127017570000,68764530
4,2022/06/24,550000,17000,3.19,534000,552000,527000,267270,144074447000,37820491500000,68764530


In [12]:
for i in tqdm(range(len(stock_df))):
    stock_df['일자'][i] = stock_df['일자'][i].replace('/', '')

100%|████████████████████████████████████████████████████████████████████████████████| 369/369 [00:07<00:00, 51.40it/s]


In [13]:
fin_stock_df = stock_df.copy()
for i in tqdm(range(len(fin_stock_df))):
    fin_stock_df.일자[i] = datetime.strptime(str(fin_stock_df.일자[i]), '%Y%m%d')   
fin_stock_df.set_index(['일자'], inplace = True)
fin_stock_df.head()

100%|████████████████████████████████████████████████████████████████████████████████| 369/369 [00:06<00:00, 57.63it/s]


,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
일자,,,,,,,,,,
2022-06-30,532000,-38000,-6.67,564000,569000,532000,460818,248496695000,36582729960000,68764530
2022-06-29,570000,-11000,-1.89,571000,579000,569000,172702,98937812000,39195782100000,68764530
2022-06-28,581000,12000,2.11,568000,581000,565000,235259,135033955000,39952191930000,68764530
2022-06-27,569000,19000,3.45,563000,577000,561000,257213,146301753000,39127017570000,68764530
2022-06-24,550000,17000,3.19,534000,552000,527000,267270,144074447000,37820491500000,68764530


## NLP처리

### 감성지수 계산

In [29]:
fin_stock_df['감성지수'] = 0
for i in tqdm(range(len(fin_stock_df))):
    if fin_stock_df['등락률'][i] > 0:
        fin_stock_df['감성지수'][i] = 1
    else:
        fin_stock_df['감성지수'][i] = 0
fin_stock_df.head()

100%|████████████████████████████████████████████████████████████████████████████████| 369/369 [00:07<00:00, 51.65it/s]


,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수,감성지수
일자,,,,,,,,,,,
2022-06-30,532000,-38000,-6.67,564000,569000,532000,460818,248496695000,36582729960000,68764530,0
2022-06-29,570000,-11000,-1.89,571000,579000,569000,172702,98937812000,39195782100000,68764530,0
2022-06-28,581000,12000,2.11,568000,581000,565000,235259,135033955000,39952191930000,68764530,1
2022-06-27,569000,19000,3.45,563000,577000,561000,257213,146301753000,39127017570000,68764530,1
2022-06-24,550000,17000,3.19,534000,552000,527000,267270,144074447000,37820491500000,68764530,1


### news_df 날짜 하루빠르게 변환

In [30]:
fin_news_df = news_df.copy()
for i in tqdm(range(len(news_df))):
    fin_news_df.Date[i] = datetime.strptime(str(news_df.Date[i]), '%Y%m%d')    
fin_news_df.head()

100%|███████████████████████████████████████████████████████████████████████| 283219/283219 [00:16<00:00, 17247.77it/s]


,Date,Title
0,2021-01-01 00:00:00,축포 터트린 코스피 새해도 랠리 이어갈까 주간전망
1,2021-01-01 00:00:00,개미가 이끈 코스피 역대 첫 GDP 넘어
2,2021-01-01 00:00:00,터키 물가 상승 억제 의지 확인에도 투자 아직 일러
3,2021-01-01 00:00:00,지난해 주식 수익률 30 안팎 달러는 마이너스
4,2021-01-01 00:00:00,표 외국환율고시표 12월 30일


In [31]:
fin_news_df['일자'] = fin_news_df['Date'] - timedelta(days=1)
fin_news_df

,Date,Title,일자
0,2021-01-01 00:00:00,축포 터트린 코스피 새해도 랠리 이어갈까 주간전망,2020-12-31
1,2021-01-01 00:00:00,개미가 이끈 코스피 역대 첫 GDP 넘어,2020-12-31
2,2021-01-01 00:00:00,터키 물가 상승 억제 의지 확인에도 투자 아직 일러,2020-12-31
3,2021-01-01 00:00:00,지난해 주식 수익률 30 안팎 달러는 마이너스,2020-12-31
4,2021-01-01 00:00:00,표 외국환율고시표 12월 30일,2020-12-31
...,...,...,...
283214,2022-06-30 00:00:00,주식 매매 상위 종목 및 환율 2022년 6월 29일자,2022-06-29
283215,2022-06-30 00:00:00,유럽증시 경기침체 공포 커지면서 하락,2022-06-29
283216,2022-06-30 00:00:00,하반기 애플 신제품 대거 출시 애플 부품주 반등 시도,2022-06-29
283217,2022-06-30 00:00:00,저축은행중앙회 이체 시 저축은행 이름 표시 추진,2022-06-29


In [32]:
fin_news_df.columns = ['Date', '제목', '일자']
fin_news_df.set_index(['일자'], inplace = True)
fin_news_df.drop(['Date'], axis=1, inplace =True)
fin_news_df

,기사제목
일자,
2020-12-31,축포 터트린 코스피 새해도 랠리 이어갈까 주간전망
2020-12-31,개미가 이끈 코스피 역대 첫 GDP 넘어
2020-12-31,터키 물가 상승 억제 의지 확인에도 투자 아직 일러
2020-12-31,지난해 주식 수익률 30 안팎 달러는 마이너스
2020-12-31,표 외국환율고시표 12월 30일
...,...
2022-06-29,주식 매매 상위 종목 및 환율 2022년 6월 29일자
2022-06-29,유럽증시 경기침체 공포 커지면서 하락
2022-06-29,하반기 애플 신제품 대거 출시 애플 부품주 반등 시도


### news_df와 stock_df 합치기

In [33]:
## 행 개수 같을때
df = fin_news_df.join(fin_stock_df, how='inner')
df

,기사제목,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수,감성지수
일자,,,,,,,,,,,,
2021-01-04,뉴욕증시 반등 성공 3대 지수 장초반 상승,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
2021-01-04,사모펀드 입시비리 정경심 항소심 재판부 배당,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
2021-01-04,삼천피 천스닥 시대 눈앞에 왔다,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
2021-01-04,개미가 끌어올린 코스피 3 000 턱밑까지 왔다,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
2021-01-04,박셀바이오 투자위험종목 지정에 6일 거래정지된다,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-29,주식 매매 상위 종목 및 환율 2022년 6월 29일자,570000,-11000,-1.89,571000,579000,569000,172702,98937812000,39195782100000,68764530,0
2022-06-29,유럽증시 경기침체 공포 커지면서 하락,570000,-11000,-1.89,571000,579000,569000,172702,98937812000,39195782100000,68764530,0
2022-06-29,하반기 애플 신제품 대거 출시 애플 부품주 반등 시도,570000,-11000,-1.89,571000,579000,569000,172702,98937812000,39195782100000,68764530,0


In [34]:
df.to_csv('NLP_data_comp_SamsungSDI.csv', encoding = 'utf-8-sig')
df.head(3)

,기사제목,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수,감성지수
일자,,,,,,,,,,,,
2021-01-04,뉴욕증시 반등 성공 3대 지수 장초반 상승,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
2021-01-04,사모펀드 입시비리 정경심 항소심 재판부 배당,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
2021-01-04,삼천피 천스닥 시대 눈앞에 왔다,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
2021-01-04,개미가 끌어올린 코스피 3 000 턱밑까지 왔다,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
2021-01-04,박셀바이오 투자위험종목 지정에 6일 거래정지된다,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1


### 감성분석 모델 적용

In [35]:
df = pd.read_csv('NLP_data_comp_SamsungSDI.csv', encoding = 'utf-8-sig', index_col=0)
df.drop_duplicates('제목', inplace = True)  # 기사제목 중복 제거
df.head()

,기사제목,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수,감성지수
일자,,,,,,,,,,,,
2021-01-04,뉴욕증시 반등 성공 3대 지수 장초반 상승,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
2021-01-04,사모펀드 입시비리 정경심 항소심 재판부 배당,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
2021-01-04,삼천피 천스닥 시대 눈앞에 왔다,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
2021-01-04,개미가 끌어올린 코스피 3 000 턱밑까지 왔다,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
2021-01-04,박셀바이오 투자위험종목 지정에 6일 거래정지된다,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1


In [36]:
# 형태소 분석을 위한 함수
def tokenizer(text):
    okt = Okt()
    return okt.morphs(text)

In [37]:
# X : title, y : price
def data_split(X, y):
    # 수집한 데이터 읽어오기
    # news_df = pd.read_excel()
    # 학습셋, 테스트셋 분리
    X_list = X.tolist()
    y_list = y.tolist()
    # 데이터의 80%는 학습셋, 20%는 테스트셋
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    return x_train, x_test, y_train, y_test

In [38]:
x_train, x_test, y_train, y_test = data_split(df['제목'], df['감성지수'])

In [39]:
vect = CountVectorizer()

In [40]:
# tfidf
tfidf = TfidfTransformer()
# tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)

In [41]:
# 로지스틱
logistic = LogisticRegression(random_state=0)    
# logistic = LogisticRegression(C=2, penalty='l2', random_state=0)     # C의 숫자가 너무 크면 과적합 (기본 1), penalty로 과적합 방지

In [47]:
pipeline = Pipeline([('vect',vect), ('tfidf',tfidf), ('clf',logistic)], verbose=True)

In [48]:
pipeline.fit(x_train, y_train)

[Pipeline] .............. (step 1 of 3) Processing vect, total=   2.9s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.1s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   3.9s


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(random_state=0))],
         verbose=True)

In [49]:
y_pred = pipeline.predict(x_test)

In [50]:
print('모델 정확도:', accuracy_score(y_test, y_pred))

모델 정확도: 0.5978957988037761


In [51]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipeline, x_train, y_train, cv=3, scoring='f1_micro')
print('평균 모델 score:', scores.mean())

[Pipeline] .............. (step 1 of 3) Processing vect, total=   2.1s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.1s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   2.6s
[Pipeline] .............. (step 1 of 3) Processing vect, total=   1.9s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.1s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   2.9s
[Pipeline] .............. (step 1 of 3) Processing vect, total=   2.1s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.1s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   3.9s
평균 모델 score: 0.5877036529927157


In [52]:
#  모델 저장
def save_model(model):
    with open('pipe.dat', 'wb') as fp:     # 쓰기, 바탕화면에 저장됨
        pickle.dump(model, fp)
    print('저장완료')     # 학습된 모델 저장 완료

In [53]:
save_model(pipeline)

저장완료


In [54]:
# 모델 사용 함수
def model_prediction():  
    # 객체를 복원, 저장된 모델 불러오기
    with open('pipe.dat','rb') as fp:     # 읽기
        pipe = pickle.load(fp)
    while True :
        text = input('뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : ')
        example = [text]
        # 예측 정확도
        r1 = np.max(pipe.predict_proba(example)*100)     # 확률값을 구해서 *100
        # 예측 결과
        r2 = pipe.predict(example)[0]     # 긍정('1'), 부정('0')
        if text == 'q':
            print("예측을 종료합니다.")
            break
        if r2 == '1':
            print(f'{stock_name} 주가가 상승할 것으로 예상됩니다.')
        else: 
            print(f'{stock_name} 주가가 하락할 것으로 예상됩니다.')
        print('확률 : %.3f' % r1)
        print('------------------------------------------------')

In [55]:
model_prediction()

뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : [속보] 코스피, 21.77포인트(0.95%) 오른 2322.11 출발
삼성SDI 주가가 하락할 것으로 예상됩니다.
확률 : 56.602
------------------------------------------------
뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 코스피 상승 출발…2,320대로 올라
삼성SDI 주가가 하락할 것으로 예상됩니다.
확률 : 53.070
------------------------------------------------
뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : LG에너지솔루션, 美배터리 공장 재검토 악재 아냐-현대차증권
삼성SDI 주가가 하락할 것으로 예상됩니다.
확률 : 77.195
------------------------------------------------
뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 글람, 실적 성장 등 체질 개선 "올해 매출 400억·영업익 100억 전망"
삼성SDI 주가가 하락할 것으로 예상됩니다.
확률 : 57.988
------------------------------------------------
뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : 삼성SDI 영업이익 상승 전망
삼성SDI 주가가 하락할 것으로 예상됩니다.
확률 : 53.681
------------------------------------------------
뉴스 타이틀을 입력해주세요(종료를 원하시면 "q"를 입력해주세요) : q
예측을 종료합니다.
